In [9]:
!pip install open_clip_torch==2.23.0 transformers==4.35.2 matplotlib

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 11.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/7.9 MB ? eta -:--:--
   ---------- ----------------------------- 2.1/7.9 MB 9.8 MB/s eta 0:00:01
   ----------------------------- ---------- 5.8/7.9 MB 13.5 MB/s eta 0:00:01
   ---------------------------------------- 7.9/7.9 MB 12.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   -------------------------------------- - 2.1/2.2 MB 13.0 MB/s eta 0:00:01
   ---------------------------------------- 2.2/2.2 MB 10.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/991.5 kB ? eta -:--:--
   --------------------------------------- 991.5/991.5 kB 11.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 2.4/2.4 MB 11.2 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
from open_clip import create_model_from_pretrained, get_tokenizer # works on open-clip-torch>=2.23.0, timm>=0.9.8

model, preprocess = create_model_from_pretrained('hf-hub:microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224')
tokenizer = get_tokenizer('hf-hub:microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224')
import torch.nn as nn
# Remplacement de la dernière couche par une couche adaptée à 8 classes
#model.visual.proj = nn.Linear(model.visual.proj.in_features, 8)

last_layer = model.visual.head[-1]  # Assuming the last layer is a linear layer

# Getting input features from the last layer
in_features = last_layer.in_features

# Replacing the last layer with a new Linear layer with 8 output features
model.visual.head[-1] = nn.Linear(in_features, 8)


C:\Users\joshu\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\joshu\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
c:\Users\joshu\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
c:\Users\joshu\AppData\Local\Programs\Python\Python311\Lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is depre

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
from transformers import CLIPProcessor, CLIPModel

In [2]:

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize
from PIL import Image
import pandas as pd
import os
from tqdm import tqdm
# Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
# 1. Dataset personnalisé
class FundusDataset(Dataset):
    def __init__(self, image_dir, csv_file, label_list, transform=None):
        self.image_dir = image_dir
        self.csv_file = csv_file
        self.labels = label_list  # Liste des pathologies
        self.df = pd.read_csv(csv_file)  # Chargement des indices
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_id = f"{self.df.iloc[idx, 0]}_244.jpg"  # Nom de l'image
        img_path = os.path.join(self.image_dir, img_id)
        # print(f"Trying to open: {img_path}")
        image = Image.open(img_path).convert("RGB")  # Chargement de l'image

        label = self.labels[idx]  # Récupération de la chaîne de pathologies

        if self.transform:
            image = self.transform(image)

        return image, label

In [4]:
# 2. Transformations des images
image_transforms = Compose([
    Resize((224, 224)),
    CenterCrop(224),
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [5]:
# 3. Chargement des données
def load_data(image_dir, csv_file, label_list, transform, batch_size=32):
    dataset = FundusDataset(image_dir=image_dir, csv_file=csv_file, label_list=label_list, transform=transform)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return loader

In [12]:
# 4. Chargement du modèle BioMedCLIP
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
model = model.to(device)

c:\Users\joshu\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\joshu\.cache\huggingface\hub\models--openai--clip-vit-base-patch32. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\Users\joshu\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers

In [13]:
# Remplacer la projection actuelle par une tête pour 8 classes
model.classification_head = nn.Sequential(
    model.visual_projection,  # Garder la projection initiale
    nn.Linear(512, 8)  # Nouvelle couche pour 8 pathologies
)
# Assurez-vous que la nouvelle sortie est utilisée dans l'entraînement
def forward(images):
    visual_features = model.vision_model(images)
    outputs = model.classification_head(visual_features)
    return outputs

In [14]:
def train(model, train_loader, val_loader, optimizer, criterion, n_epochs):
    model.train()

    for epoch in range(n_epochs):
        train_loss = 0.0

        for images, labels in tqdm(train_loader):
            images = images.to(device)

            # Concaténer les tenseurs de labels en un seul grand tenseur (batch_size, 32)
            labels_tensor = torch.stack(labels).to(device)

            # Vérification de la forme des labels après concaténation
            #print(f"Labels après concaténation: {labels_tensor.shape}")

            # Zero gradients
            optimizer.zero_grad()

            # Forward pass via la nouvelle tête de classification
            visual_features = model.vision_model(pixel_values=images).pooler_output
            outputs = model.classification_head(visual_features)

            outputs = outputs.transpose(0, 1)
            labels_tensor = labels_tensor.float()  # Convertir en float
            # Vérification des dimensions des sorties et des labels
            if labels_tensor.shape != outputs.shape:
                print(f"Shape mismatch: labels_tensor.shape = {labels_tensor.shape}, outputs.shape = {outputs.shape}")
                continue  # Passez à l'itération suivante si les formes ne correspondent pas

            # Calcul de la perte multi-label
            loss = criterion(outputs, labels_tensor)

            # Backward pass
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        # Affichage de la perte moyenne par époque
        print(f"Epoch {epoch + 1}/{n_epochs}, Loss: {train_loss / len(train_loader)}")

        # Sauvegarde du modèle à la fin de chaque époque
        save_dir = "/content/drive/MyDrive/RIADD"
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        model_path = os.path.join(save_dir, f"biomedclip_epoch_{epoch + 1}.pt")
        torch.save(model.state_dict(), model_path)
        print(f"Modèle sauvegardé : {model_path}")


In [15]:
# 6. Optimiseur et fonction de perte
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
#criterion = torch.nn.CrossEntropyLoss()
criterion = torch.nn.BCEWithLogitsLoss()

In [ ]:


# 7. Préparer les loaders
train_image_dir = "/content/drive/MyDrive/RIADD/Training_244"
val_image_dir = "/content/drive/MyDrive/RIADD/Validation_244"
test_image_dir = "/content/drive/MyDrive/RIADD/Test_244"

train_csv = "/content/drive/MyDrive/RIADD/RFMiD_Training.csv"
val_csv = "/content/drive/MyDrive/RIADD/RFMiD_Validation.csv"
test_csv = "/content/drive/MyDrive/RIADD/RFMiD_Testing.csv"

In [15]:
csv_train = pd.read_csv(train_csv)
csv_test = pd.read_csv(test_csv)
csv_val = pd.read_csv(val_csv)
# Get the column names for the labels (excluding the 'ID' column)
label = csv_train.columns[1:]

label_list_train = [
    vect for vect in [[csv_train[c][i] for c in csv_train.columns[1:]] for i in range(len(csv_train))]
]
label_list_test = [
    vect for vect in [[csv_test[c][i] for c in csv_test.columns[1:]] for i in range(len(csv_test))]
]
label_list_val = [
    vect for vect in [[csv_val[c][i] for c in csv_val.columns[1:]] for i in range(len(csv_val))]
]

In [16]:
print(label_list_val)
print(type(label_list_val[0][0]))

[[0, 1, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 1], [0, 1, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 1], [0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 1, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 1], [0, 1, 1, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 1, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0

In [17]:
import os
folder_path = '/content/drive/MyDrive/RIADD/Training_244'
print(os.listdir(folder_path))
print('1080_244.jpg' in os.listdir(folder_path))
print(folder_path)

['1827_244.jpg', '1831_244.jpg', '1823_244.jpg', '183_244.jpg', '1821_244.jpg', '1848_244.jpg', '1838_244.jpg', '1832_244.jpg', '1835_244.jpg', '1837_244.jpg', '184_244.jpg', '1828_244.jpg', '1833_244.jpg', '1826_244.jpg', '1836_244.jpg', '1824_244.jpg', '1834_244.jpg', '186_244.jpg', '1853_244.jpg', '1857_244.jpg', '1849_244.jpg', '1845_244.jpg', '1850_244.jpg', '1847_244.jpg', '1851_244.jpg', '187_244.jpg', '1863_244.jpg', '1860_244.jpg', '1846_244.jpg', '1852_244.jpg', '185_244.jpg', '1844_244.jpg', '1861_244.jpg', '1843_244.jpg', '1859_244.jpg', '1869_244.jpg', '1870_244.jpg', '1865_244.jpg', '1862_244.jpg', '1840_244.jpg', '1866_244.jpg', '1864_244.jpg', '1868_244.jpg', '1855_244.jpg', '1842_244.jpg', '1867_244.jpg', '1858_244.jpg', '1856_244.jpg', '1854_244.jpg', '1877_244.jpg', '1884_244.jpg', '1872_244.jpg', '188_244.jpg', '1882_244.jpg', '1873_244.jpg', '1889_244.jpg', '1887_244.jpg', '1876_244.jpg', '1883_244.jpg', '1881_244.jpg', '1885_244.jpg', '1886_244.jpg', '1871_244.jpg

In [16]:
# Ajout de la tête de classification si elle n'existe pas encore
if not hasattr(model, 'classification_head'):
    model.classification_head = nn.Sequential(
        model.visual_projection,  # Utiliser la projection existante
        nn.Linear(512, 8)  # Adapter à 8 pathologies
    )

In [ ]:
# 8. Lancement de l'entraînement
model_path = '/content/drive/MyDrive/RIADD/biomedclip_epoch_10.pt'
# Create the data loaders using the load_data function
model.load_state_dict(torch.load(model_path))
train_loader = load_data(train_image_dir, train_csv, label_list_train, image_transforms) # Create train_loader
val_loader = load_data(val_image_dir, val_csv, label_list_val, image_transforms) # Create val_loader

#model.load_state_dict(torch.load(model_path))
model.to(device)
train(model, train_loader, val_loader, optimizer, criterion, n_epochs=10)

100%|██████████| 60/60 [00:28<00:00,  2.08it/s]


Epoch 1/10, Loss: 0.38498498847087226
Modèle sauvegardé : /content/drive/MyDrive/RIADD/biomedclip_epoch_1.pt


100%|██████████| 60/60 [00:28<00:00,  2.09it/s]


Epoch 2/10, Loss: 0.35902111132939657
Modèle sauvegardé : /content/drive/MyDrive/RIADD/biomedclip_epoch_2.pt


100%|██████████| 60/60 [00:29<00:00,  2.03it/s]


...

Epoch 8/10, Loss: 0.3093821013967196
Modèle sauvegardé : /content/drive/MyDrive/RIADD/biomedclip_epoch_8.pt


100%|██████████| 60/60 [00:29<00:00,  2.01it/s]


Epoch 9/10, Loss: 0.30088013783097267
Modèle sauvegardé : /content/drive/MyDrive/RIADD/biomedclip_epoch_9.pt


100%|██████████| 60/60 [00:29<00:00,  2.06it/s]


Epoch 10/10, Loss: 0.2995141456524531
Modèle sauvegardé : /content/drive/MyDrive/RIADD/biomedclip_epoch_10.pt


In [6]:

test_image_dir = "C:/Users/joshu/Desktop/Cours IMT/2A/Commande entreprise/Base de données/RIADD/Test/Test 244"

test_csv = "C:/Users/joshu/Desktop/Cours IMT/2A/Commande entreprise/Base de données/RIADD/RFMiD_Testing.csv"
csv_test = pd.read_csv(test_csv)
label_list_test = [
    vect for vect in [[csv_test[c][i] for c in csv_test.columns[1:]] for i in range(len(csv_test))]
]

In [21]:
import torch
from tqdm import tqdm

def evaluate(model, test_loader):
    model.eval()  # Mode évaluation
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in tqdm(test_loader):
            images = images.to(device)

            # Conversion des labels en tenseur float32
            # labels_tensor = torch.stack(labels).float().to(device)
            labels_tensor = torch.stack(labels).to(device)
            
            # Forward pass via la nouvelle tête de classification
            visual_features = model.vision_model(pixel_values=images).pooler_output
            outputs = model.classification_head(visual_features)

            outputs = outputs.transpose(0, 1)
            labels_tensor = labels_tensor.float()  # Convertir en float
            # Application d'un seuil pour obtenir des prédictions binaires
            predictions = (outputs > 0).float()

            # Calcul du nombre d'exactitudes (comparaison des prédictions aux labels)
            correct += (predictions == labels_tensor).all(dim=1).sum().item()
            total += labels_tensor.size(0)

    accuracy = correct / total * 100
    print(f"Accuracy: {accuracy:.2f}%")

# Appeler la fonction avec votre test_loader
test_loader = load_data(test_image_dir, test_csv, label_list_test, image_transforms)
# 8. Lancement de l'entraînement
model_path = 'C:/Users/joshu/Desktop/Cours IMT/2A/Commande entreprise/Base de données/RIADD/biomedclip_epoch_10.pt'
# Create the data loaders using the load_data function
model.load_state_dict(torch.load(model_path))
model.to(device)
evaluate(model, test_loader)


C:\Users\joshu\AppData\Local\Temp\ipykernel_5616\3149976047.py:37: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))
100%|█████████

Accuracy: 15.62%


In [28]:
dir = "C:/Users/joshu/Desktop/Cours IMT/2A/Commande entreprise/Base de données/RIADD/Test/Test 244/1_244.jpg"
label_0_test = [
    [csv_test[c][0] for c in csv_test.columns[1:]]
]
essai_loader = load_data(dir, test_csv, label_0_test, image_transforms)
for images, labels in test_loader:
    images = images.to(device)
    labels_tensor = torch.stack(labels).to(device)
    visual_features = model.vision_model(pixel_values=images).pooler_output
    outputs = model.classification_head(visual_features)
    outputs = outputs.transpose(0, 1)
    labels_tensor = labels_tensor.float()
    predictions = (outputs > 0).float()
    print(predictions)
    print(labels_tensor)
    break

tensor([[0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1.,
         1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 1., 0., 0., 1., 

In [ ]:
from IPython.display import display, HTML
from transformers import CLIPTokenizer
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")
def evaluate(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    all_predictions = []  # Store all predictions
    all_labels = []  # Store all actual labels
    with torch.no_grad():
        for images, labels in tqdm(test_loader):
            images = images.to(device)
            text_inputs = processor(text=labels, return_tensors="pt", padding=True).to(device)
            outputs = model(pixel_values=images, input_ids=text_inputs.input_ids, attention_mask=text_inputs.attention_mask)
            logits_per_image = outputs.logits_per_image

            predictions = logits_per_image.argmax(dim=1)
            correct += (predictions == torch.arange(len(labels)).to(device)).sum().item()
            total += len(labels)

            # Store predictions and labels
            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(labels)

    print(f"Accuracy: {correct / total * 100:.2f}%")
    # Display predictions and actual labels
    for i in range(len(all_predictions)):
        predicted_label = tokenizer.decode(all_predictions[i], skip_special_tokens=True) # Decode prediction
        display(HTML(f"<b>Prediction:</b> {predicted_label}<br><b>Actual Label:</b> {all_labels[i]}"))
    # # Display predictions and actual labels
    # for i in range(len(all_predictions)):
    #     display(HTML(f"<b>Prediction:</b> {all_predictions[i]}<br><b>Actual Label:</b> {all_labels[i]}"))

test_loader = load_data(test_image_dir, test_csv, label_list_test, image_transforms)
evaluate(model, test_loader)